In [2]:
cd ../../..

/home/cridonoso/Documents/astromer


/home/cridonoso/miniconda3/envs/tf2.14/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [106]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import pandas as pd
import polars as pl
import os
import toml



In [ ]:
gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)

In [74]:
# List files in the specified folder
folder_id = 'root'
file_list = drive.ListFile({'includeItemsFromAllDrives':True,
                            'driveId':'0ANj7Lpiu7QacUk9PVA',
                            'corpora':'drive',
                            'supportsAllDrives':True,
                            # This is the ID of the parent folder. It can be becked on the webpage or via another query
                            'q': "'1HaRYj6lA2uIS4poQw2yIZYZaRTmD82Wu' in parents and trashed=false"}).GetList()

In [75]:
ids = [{'name': file['title'], 'id': file['id']} for file in file_list]

In [76]:
root = './data/temp/'
os.makedirs(os.path.join(root, 'light_curves'), exist_ok=True)
for i in ids:
    file_list = drive.ListFile({'includeItemsFromAllDrives':True,
                            'driveId':'0ANj7Lpiu7QacUk9PVA',
                            'corpora':'drive',
                            'supportsAllDrives':True,
                            'q': "'{}' in parents and trashed=false".format(i['id'])}).GetList()

    for i, file in enumerate(file_list):    
        if 'metadata' in file['originalFilename']:
            target_file = os.path.join(root, 'metadata.parquet')
        else:
            name = file['originalFilename'].split('.parquet')[0]
            target_file = os.path.join(root, 'light_curves', name)
        print(target_file)
        file.GetContentFile(target_file)
        if i == 3:
            break
    break

./data/temp/metadata.parquet
./data/temp/light_curves/Group_4_part.2
./data/temp/light_curves/Group_4_part.5
./data/temp/light_curves/Group_4_part.4


In [135]:
observations = pl.read_parquet('./data/temp/light_curves/*')
metadata     = pd.read_parquet('./data/temp/metadata.parquet')

In [136]:
metadata

,newID,Field,Tile,Star_id,shard,Class,ID,Band,Label


In [99]:
new_meta = metadata.drop_duplicates('newID')
del new_meta['Band']
cardinality = metadata.groupby(['newID']).count()['Band'].reset_index()
new_meta = pd.merge(new_meta, cardinality, on='newID', how='inner')
new_meta['Label'] = pd.Categorical(new_meta['Class']).codes


In [123]:
to_change = {}
for index, u in enumerate(new_meta['shard'].unique()):
    shard_name = str(index).rjust(3, '0')
    to_change[u] = 'shard_{}'.format(shard_name)

new_meta['shard'] = new_meta['shard'].replace(to_change)

0         shard_000
1         shard_000
2         shard_000
3         shard_000
4         shard_000
            ...    
550040    shard_041
550041    shard_041
550042    shard_041
550043    shard_041
550044    shard_041
Name: shard, Length: 550045, dtype: string

In [116]:
target_folder = "./data/records/bigmacho/{}".format(ids[0]['name'])
config = {
    "id_column": {
        "value": "newID",
        "dtype": "integer",
    },
    "target": {
        "path": target_folder,
        "dtype": "string"
    },
    "context_features": {
        'path': "./data/temp/metadata.parquet",
        'dtype': "string",
        'value': [ "ID", "Class", "Band", "Label", "shard",],
        'dtypes': [ "string", "string", "integer", "integer", "string",]
    },
    "sequential_features": {
        'path': "./data/temp/light_curves",
        'value': [ "observation_date", "mag", "err"],
        'dtypes': [ "float", "float", "float"],
    }
}

In [117]:
os.makedirs(target_folder, exist_ok=True)
with open(os.path.join(target_folder, 'config.toml'), "w") as f:
    toml.dump(config, f)

In [118]:
target_folder

'./data/records/bigmacho/F_309'